<a href="https://colab.research.google.com/github/imranrahman1997/Machine-Learning_projects/blob/master/plant_disease_fresh%20code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:

#before importing the dataset we want to use this code
# The Kaggle API client expects this file to be in ~/.kaggle,
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d noulam/tomato

In [ ]:
from zipfile import ZipFile

file_name = "/content/tomato.zip"

with ZipFile(file_name, "r") as zip:
  zip.extractall()
  print("done")

In [ ]:
!pip install tensorflow-gpu

In [ ]:
!nvidia-smi

In [ ]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [ ]:
IMAGE_SIZE = [224, 224]

train_path = "/content/Datasets/train"
test_path = "/content/Datasets/test"

In [ ]:
inception = InceptionV3(input_shape = IMAGE_SIZE + [3], weights='imagenet', include_top= False)

In [ ]:
for layer in inception.layers:
  layer.trainable = False

In [ ]:
#NUMBER of out put classes
folders = glob("/content/Datasets/train/*")

In [ ]:
#our flatten layer
x = Flatten()(inception.output)

In [ ]:
import tensorflow as tf
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
      if(logs.get('accuracy')>0.97):
        print("\nReached 60% accuracy so cancelling training!")
        self.model.stop_training = True

callbacks = myCallback()

In [ ]:
prediction = Dense(len(folders), activation = 'softmax')(x)

#creat model object
model = Model(inputs = inception.input , outputs = prediction)

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [ ]:
#use image generator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range  = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory('/content/Datasets/train',
                                                 target_size = (224, 224),
                                                 batch_size = 18,
                                                 class_mode = 'categorical')

In [ ]:
test_set = test_datagen.flow_from_directory("/content/Datasets/test",
                                           target_size  = (224, 224),
                                           batch_size = 18,
                                           class_mode = "categorical")

In [ ]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=70,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set),
  callbacks=[callbacks]
)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
from tensorflow.keras.models import load_model
model.save("model_inception.h5")

In [ ]:
y_pred = model.predict(test_set)

In [ ]:

y_pred

In [ ]:
import numpy as np
y_pred = np.argmax(y_pred, axis = 1)

In [ ]:
y_pred

array([1, 4, 9, ..., 8, 9, 1])

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(224, 224,3))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images)


  print(classes)

classes2 = np.argmax(classes, axis=1)

if classes2[0] == 0:
        print('it is Tomato_Yellow_Leaf_Curl_Virus')
elif classes2[0] == 1:
        print('It is Tomato_mosaic_virus')
elif classes2[0] == 2:
        print('It is a Septoria_leaf_spot')
elif classes2[0] == 3:
        print('It is a Spider_mites')
elif classes2[0] == 4:
        print('It is a Late_blight')
elif classes2[0] == 5:
        print('It is a healthy')
elif classes2[0] == 6:
        print('It is a Early_blight')
elif classes2[0] == 7:
        print('It is a Bacterial_spot')
elif classes2[0] == 8:
        print('It is a Leaf_Mold')
elif classes2[0] == 9:
        print('It is a Target_Spot')
else:
        print('Can\'t recognize the image')

print('done')

In [ ]:
folders

In [ ]:
test_set

In [ ]:
class_dict = training_set.class_indices
class_dict

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image


uploaded = files.upload()

for fn in uploaded.keys():

  # predicting images
  path =  fn
  img = image.load_img(path, target_size=(224, 224,3))
  x = image.img_to_array(img)
  x = x / 255
  x = x.reshape(-1, 224, 224, 3)
  images = np.vstack([x])
  prediction = model.predict(images)


  print(prediction)



if np.argmax(prediction) == 0:
    print("Tomato___Bacterial_spot")
elif np.argmax(prediction) == 1:
    print("Tomato___Early_Blight")
elif np.argmax(prediction) == 2:
    print("Tomato___Late Blight")
elif np.argmax(prediction) == 3:
    print("Tomato___Leaf Mold")
elif np.argmax(prediction) == 4:
    print("Tomato___Septoria Leaf Spot")
elif np.argmax(prediction) == 5:
    print("Tomato___Spider mites")
elif np.argmax(prediction) == 6:
    print("Tomato___Target Spot")
elif np.argmax(prediction) == 7:
    print("Tomato___Yellow Leaf Curl Virus")
elif np.argmax(prediction) == 8:
    print("Tomato___Mosaic Virus")
else:
    print("Tomato___Healthy")



In [ ]:
model.save("model_inception.h5")